In [ ]:
! pip install load_dotenv

In [ ]:
! pip install langchain-openai

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

### Componentes de LangChain

Prompts

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Dime un título {adjective} para una película de {content}."
)
prompt_template.format(adjective="gracioso", content="acción")

'Dime un título gracioso para una película de acción.'

In [12]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente de IA. Tu nombre es {name}."),
        ("human", "¿Cómo estás?"),
        ("ai", "Estoy bien, gracias."),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="María", user_input="¿Cómo te llamas?")

In [9]:
messages

[SystemMessage(content='Eres un asistente de IA. Tu nombre es Bob.'),
 HumanMessage(content='¿Cómo estás?'),
 AIMessage(content='Estoy bien, gracias.'),
 HumanMessage(content='¿Cómo te llamas?')]

Chat Models

In [10]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [17]:
messages = chat_template.format_messages(name="María", user_input="¿Crea una canción sobre la Eurocopa?")

In [18]:
chat.invoke(messages)

AIMessage(content='Claro, aquí tienes una canción improvisada sobre la Eurocopa:\n\n(Verse 1)\nEn el campo de fútbol brillan las estrellas,\nLa emoción se siente en cada esquina,\nLas selecciones luchan con fuerza y destreza,\nPor levantar la copa, esa es su rutina.\n\n(Coro)\nEurocopa, pasión y emoción,\nGoles y victorias en cada ocasión,\nLa afición se une con un solo corazón,\nCelebrando juntos esta competición.\n\n(Verse 2)\nLos jugadores corren tras el balón,\nDefendiendo su país con honor,\nEl balón rueda de un lado a otro sin cesar,\nEn busca del gol que haga vibrar.\n\n(Coro)\nEurocopa, pasión y emoción,\nGoles y victorias en cada ocasión,\nLa afición se une con un solo corazón,\nCelebrando juntos esta competición.\n\n(Estribillo)\nEurocopa, fiesta de fútbol y pasión,\nUniendo países en cada canción,\nQue retumbe el estadio con alegría,\nPorque en la Eurocopa la emoción nunca se olvida.\n\nEspero que te haya gustado la canción. ¡Vamos Eurocopa!', additional_kwargs={'refusal': N

In [ ]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

Output Parsers

In [ ]:
! pip install langchain

In [32]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

# Inicializar el modelo
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# Definir la estructura de datos deseada
class BookInfo(BaseModel):
    title: str = Field(description="El título del libro")
    author: str = Field(description="El autor del libro")
    publication_year: int = Field(description="El año de publicación del libro")
    description: str = Field(description="Una breve descripción del libro")

    # Validación personalizada: El año de publicación no debe ser en el futuro
    @validator("publication_year")
    def year_not_in_future(cls, field):
        if field > 2024:  # Asumiendo que el año actual es 2024
            raise ValueError("El año de publicación no puede ser en el futuro.")
        return field

# Configurar el parser e inyectar instrucciones en la plantilla del prompt
parser = PydanticOutputParser(pydantic_object=BookInfo)

prompt = PromptTemplate(
    template="Dime información sobre un libro.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Crear la consulta para solicitar al modelo que rellene la estructura de datos
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Dame información sobre el libro 'Cien años de soledad'."})
book_info = parser.parse(output)

# Mostrar el resultado estructurado
print(book_info)


title='Cien años de soledad' author='Gabriel García Márquez' publication_year=1967 description='Cien años de soledad es una novela del escritor colombiano Gabriel García Márquez, publicada en 1967. Es considerada una obra maestra de la literatura hispanoamericana y una de las obras más importantes del siglo XX. La historia narra la vida de la familia Buendía en el pueblo ficticio de Macondo, a lo largo de siete generaciones. A través de la magia y el realismo mágico, García Márquez explora temas como el amor, la soledad, la muerte y la identidad latinoamericana.'


Ejemplo de Chain Uniendo varios componentes

In [33]:
book_info = prompt | model | parser

# Mostrar el resultado estructurado
print(book_info.invoke({"query": "Dame información sobre el libro 'Cien años de soledad'."}))

title='Cien años de soledad' author='Gabriel García Márquez' publication_year=1967 description='Cien años de soledad es una novela del escritor colombiano Gabriel García Márquez, publicada en 1967. Es considerada una obra maestra de la literatura hispanoamericana y una de las obras más importantes del siglo XX. La historia narra la vida de la familia Buendía en el pueblo ficticio de Macondo, a lo largo de siete generaciones. A través de la magia y el realismo mágico, García Márquez explora temas como el amor, la soledad, la muerte y la identidad latinoamericana.'


LCEL y Chains en LangChain

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

# Definir un prompt y un modelo
prompt = PromptTemplate.from_template("What is the capital of {country}?")
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# Crear una chain combinando el prompt y el modelo
chain = prompt | model

# Ejecutar la chain con un input específico
result = chain.invoke({"country": "France"})
print(result)  # Debería imprimir: "The capital of France is Paris."




The capital of France is Paris.


In [ ]:
! pip install langchain-community

In [37]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# Definir el prompt para la cadena
prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)

# Definir el modelo de lenguaje
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Crear la chain que combina documentos
chain = create_stuff_documents_chain(llm, prompt)

# Crear documentos de ejemplo
docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content="Jamal loves green but not as much as he loves orange")
]

# Ejecutar la chain con los documentos
result = chain.invoke({"context": docs})
print(result)  # Debería imprimir un resumen de los colores favoritos.


c:\Users\Diana\Desktop\azure-openai-modelos-ia-generativa-3351165\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




Jesse: Red
Jamal: Orange
